In [1]:
#Importing libraries
import pandas as pd
from google.cloud import bigquery
import json
import os 


In [2]:
#Obtain path to BQ credentiasls.
with open('path.txt', 'r') as f:
    path = f.read()
#print(path)

In [3]:
#Using credentials to connect to BQ
# Set enviroment variable GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=path

In [4]:
#Let's initialize a bq client.
#This dataset comes from a BQ-DB. Project's name is power-bi*
BigQuery_client = bigquery.Client()
query="""
#standardSQL
SELECT * FROM `power-bi-326113.analytics_264987177.events_*`
"""
#Save results in query_results
query_results = BigQuery_client.query(query)

In [5]:
#Transform query_results to a pandas dataframe.
df = pd.read_gbq(query)
print(df.head(5))

  event_date   event_timestamp       event_name  \
0   20211004  1633384276241364      first_visit   
1   20211004  1633384276241364    session_start   
2   20211004  1633384276241364        page_view   
3   20211004  1633384361840593           scroll   
4   20211004  1633384400027467  user_engagement   

                                        event_params  \
0  [{'key': 'page_location', 'value': {'string_va...   
1  [{'key': 'page_location', 'value': {'string_va...   
2  [{'key': 'source', 'value': {'string_value': '...   
3  [{'key': 'page_location', 'value': {'string_va...   
4  [{'key': 'medium', 'value': {'string_value': '...   

   event_previous_timestamp  event_value_in_usd  event_bundle_sequence_id  \
0                       NaN                 NaN                -376362028   
1                       NaN                 NaN                -376362028   
2                       NaN                 NaN                -376362028   
3                       NaN                 NaN 

In [13]:
#So, that was the hard part. Now, we need to clean this dataset and get some important data such as city and country.
#I'm going to use a list comprehension to do that, then I'll use some high level functions like map. I think both are very useful and very helpful to learn.
df['city'] = [x['city'] for x in df['geo']]
#df['country'] = [x['geo']['country'] for x in df['geo']]
df['country'] = df['geo'].map(lambda x: x['country'])



In [12]:
df.head(5)

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,city,country
0,20211004,1633384276241364,first_visit,"[{'key': 'page_location', 'value': {'string_va...",NaN,NaN,-376362028,NaN,None,1252547838.1633384275,...,"{'continent': 'Americas', 'country': 'Argentin...",None,{'name': 'MemberCustomField_Soluciona_tu_deuda...,2366041961,WEB,None,None,[],Rio Grande,Argentina
1,20211004,1633384276241364,session_start,"[{'key': 'page_location', 'value': {'string_va...",NaN,NaN,-376362028,NaN,None,1252547838.1633384275,...,"{'continent': 'Americas', 'country': 'Argentin...",None,{'name': 'MemberCustomField_Soluciona_tu_deuda...,2366041961,WEB,None,None,[],Rio Grande,Argentina
2,20211004,1633384276241364,page_view,"[{'key': 'source', 'value': {'string_value': '...",NaN,NaN,-376362028,NaN,None,1252547838.1633384275,...,"{'continent': 'Americas', 'country': 'Argentin...",None,{'name': 'MemberCustomField_Soluciona_tu_deuda...,2366041961,WEB,None,None,[],Rio Grande,Argentina
3,20211004,1633384361840593,scroll,"[{'key': 'page_location', 'value': {'string_va...",NaN,NaN,-290762799,NaN,None,1252547838.1633384275,...,"{'continent': 'Americas', 'country': 'Argentin...",None,{'name': 'MemberCustomField_Soluciona_tu_deuda...,2366041961,WEB,None,None,[],Rio Grande,Argentina
4,20211004,1633384400027467,user_engagement,"[{'key': 'medium', 'value': {'string_value': '...",NaN,NaN,-252575925,NaN,None,1252547838.1633384275,...,"{'continent': 'Americas', 'country': 'Argentin...",None,{'name': 'MemberCustomField_Soluciona_tu_deuda...,2366041961,WEB,None,None,[],Rio Grande,Argentina
